## Import Modules

In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [21]:
#Gather Data
boston_dataset= load_boston()
data=pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features=data.drop(['INDUS', 'AGE'], axis=1)

log_prices=np.log(boston_dataset.target)
target=pd.DataFrame(log_prices, columns=['PRICE'])


C:\Users\HP USER\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset

In [58]:
CRIME_IDX=0
ZN_IDX=1
CHAS_IDX=2
RM_IDX=4
PTRATIO_IDX=8

# property_stats=np.ndarray(shape=(1,11))
# property_stats[0][CRIME_IDX]=features['CRIM'].mean()
# property_stats[0][ZN_IDX]=features['ZN'].mean()
# property_stats[0][CHAS_IDX]=features['CHAS'].mean()


property_stats =features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [37]:
features.mean()





CRIM         3.613524
ZN          11.363636
CHAS         0.069170
NOX          0.554695
RM           6.284634
DIS          3.795043
RAD          9.549407
TAX        408.237154
PTRATIO     18.455534
B          356.674032
LSTAT       12.653063
dtype: float64

In [38]:
type(features.mean())



pandas.core.series.Series

In [39]:
features.mean().shape

(11,)

In [40]:
features.mean().values

array([3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
       6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
       1.84555336e+01, 3.56674032e+02, 1.26530632e+01])

In [41]:
features.mean().values.shape

(11,)

In [42]:
features.mean().values.reshape(1,11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [51]:
regr=LinearRegression().fit(features, target)
fitted_val=regr.predict(features)

#calculate the MSE and RMSE using sklearn
MSE= mean_squared_error(target, fitted_val)
RMSE=np.sqrt(MSE)
RMSE

0.18751213519713034

In [64]:
def get_log_estimate(nr_rooms, students_per_classroom, next_to_river=False, high_confidence=True):
    #configure propertY
    property_stats[0][RM_IDX]=nr_rooms
    property_stats[0][PTRATIO_IDX]=students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX]=1
    else:
        property_stats[0][CHAS_IDX]=0
        
    
    #make prediction
    log_estimate=regr.predict(property_stats)[0][0]
    
    if high_confidence:
        upper_bound=log_estimate + 2*RMSE
        lower_bound=log_estimate - 2*RMSE
        interval=95
        
    else:
        upper_bound=log_estimate + RMSE
        lower_bound=log_estimate - RMSE
        interval=68
    
    return log_estimate, upper_bound, lower_bound, interval

In [68]:
get_log_estimate(3, 20, next_to_river=True ) #, high_confidence=False--if you what a narrow instead of a wide range

C:\Users\HP USER\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.7767581914803996, 3.15178246187466, 2.401733921086139, 95)

In [69]:
np.median(boston_dataset.target)

21.2

In [72]:
current_median_price=583.3 #Zillion estimate
SCALE_FACTOR= current_median_price/ np.median(boston_dataset.target)
SCALE_FACTOR

log_est, upper, lower, conf=get_log_estimate(9, students_per_classroom=15, next_to_river=False, high_confidence=False)

#convert to todays dollars
dollar_est=np.e**log_est * 1000 *SCALE_FACTOR
dollar_hi=np.e**upper * 1000 *SCALE_FACTOR
dollar_low=np.e**lower * 1000 *SCALE_FACTOR

#Round the dollar values to nearest thousand
rounded_est= np.around(dollar_est, -3)
rounded_hi= np.around(dollar_hi, -3)
rounded_low= np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidencee the valuation range is ')
print(f'{rounded_low} at the lower end to USD {rounded_hi} at the high end')



The estimated property value is 827000.0.
At 68% confidencee the valuation range is 
685000.0 at the lower end to USD 997000.0 at the high end


C:\Users\HP USER\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [82]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    
    """teh offic
    """
    
    if rm < 1 or ptratio <1:
        print('that is unrealistic: try again')
        return

    log_est, upper, lower, conf=get_log_estimate(rm, ptratio, next_to_river=chas, high_confidence=large_range)

    #convert to todays dollars
    dollar_est=np.e**log_est * 1000 *SCALE_FACTOR
    dollar_hi=np.e**upper * 1000 *SCALE_FACTOR
    dollar_low=np.e**lower * 1000 *SCALE_FACTOR

    #Round the dollar values to nearest thousand
    rounded_est= np.around(dollar_est, -3)
    rounded_hi= np.around(dollar_hi, -3)
    rounded_low= np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidencee the valuation range is ')
    print(f'{rounded_low} at the lower end to USD {rounded_hi} at the high end')

In [81]:
get_dollar_estimate(rm=2, ptratio=30, chas=True)

The estimated property value is 278000.0.
At 95% confidencee the valuation range is 
191000.0 at the lower end to USD 404000.0 at the high end


C:\Users\HP USER\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
